In [5]:
import wave
import numpy as np
import IPython.display as ipd

In [6]:
def encode_message(file, secret="An eye for an eye makes the whole world blind"):
    sound = wave.open(file, mode='rb')
    
    # Convert audio frames to a byte array
    samples = bytearray(list(sound.readframes(sound.getnframes())))
    
    # Append dummy data
    secret_with_dummy_data = secret + int((len(samples) - (len(secret)*8*8))/8) * '#'
    
    # Convert to bit array
    bits = list(map(int, ''.join([bin(ord(i)).lstrip('0b').rjust(8, '0') for i in secret_with_dummy_data])))
    
    # Encode message into the LSB of audio samples
    for i, bit in enumerate(bits):
        samples[i] = (samples[i] & 254) | int(bit)
        
    modified_samples = bytes(samples)
    # Save the modified audio
    with wave.open('sound_with_secret.wav', 'wb') as record:
        record.setparams(sound.getparams())
        record.writeframes(modified_samples)
    sound.close()

In [7]:
encode_message('./Ex3_sound5.wav')

In [8]:
# Message is hidden within audio. There is a noise on the audio which represents the manipulation of the bits.
ipd.Audio("./Ex3_sound5.wav")

In [9]:
ipd.Audio("./sound_with_secret.wav")

In [55]:
def decode_message(file):
    sound = wave.open(file, mode='rb')
    
    # Convert audio frames to a byte array
    samples = bytearray(list(sound.readframes(sound.getnframes())))
    
    # Extract LSB on each byte
    extracted_lsb = [samples[i] & 1 for i in range(len(samples))]
    
    string_lsb = "".join(chr(int("".join(map(str,extracted_lsb[i:i+8])),2)) for i in range(0,len(extracted_lsb),8))
    
    # Cut the filter characters
    decoded_message = string_lsb.split("###")[0]
    print(decoded_message)
    
    #print(f"Decoded message: {decoded_message}")
    sound.close()
    return

In [56]:
decode_message('./sound_with_secret.wav')

An eye for an eye makes the whole world blind
